# Example 4: Is Olympic Success Statistically Predictable?

Tasks:
* Hypothesis Test
    * H0: GDP has no relationship with medal count
    * H1: GDP is associated with medal count


* Correlation Tests
* Simple linear Regression

* Outputs
    * Coefficient interpretataion
    * R2 discussion
    * limitations (small sample size)

In [43]:
#Imports
import pandas as pd
import plotly.express as px
import numpy as np
import statsmodels.api as sm

In [44]:
file_path = 'C:/Users/viole/dev/analytics/kaggle/olympics-data-analysis/data/olympic_countries_efficiency.csv'
df = pd.read_csv(file_path)
df.head()

,NOC,ISO3,Year,population,gdp_per_capita,income_group,host_country,athletes_sent,sports_participated,events_participated,female_athlete_percentage,prev_total_medals,prev_medals_per_athlete,Gold,Silver,Bronze,total_medals,medals_per_athlete
0,AFG,AFG,2004,23560654.0,221.763654,Low income,0,5,4,5,40.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
1,AFG,AFG,2008,26482622.0,381.733238,Low income,0,4,2,4,25.000000,0.0,0.000000,0.0,0.0,1.0,1.0,0.250000
2,AFG,AFG,2012,30560034.0,651.417134,Low income,0,6,4,6,16.666667,1.0,0.250000,0.0,0.0,1.0,1.0,0.166667
3,AFG,AFG,2016,34700612.0,522.082216,Low income,0,3,2,3,33.333333,1.0,0.166667,0.0,0.0,0.0,0.0,0.000000
4,ALB,ALB,1992,3247039.0,200.852220,Low income,0,7,4,8,22.222222,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [45]:
df.columns

Index(['NOC', 'ISO3', 'Year', 'population', 'gdp_per_capita', 'income_group',
       'host_country', 'athletes_sent', 'sports_participated',
       'events_participated', 'female_athlete_percentage', 'prev_total_medals',
       'prev_medals_per_athlete', 'Gold', 'Silver', 'Bronze', 'total_medals',
       'medals_per_athlete'],
      dtype='str')

## Columns Needed

* NOC
* Year
* population
* gdp_per_capita
* total_medals
* gdp (calculated)

In [46]:
# Keep only columns needed, create a copy of df
df_small = df[[
    'NOC',
    'Year',
    'population',
    'gdp_per_capita',
    'total_medals'
]].copy()

df_small.head()

,NOC,Year,population,gdp_per_capita,total_medals
0,AFG,2004,23560654.0,221.763654,0.0
1,AFG,2008,26482622.0,381.733238,1.0
2,AFG,2012,30560034.0,651.417134,1.0
3,AFG,2016,34700612.0,522.082216,0.0
4,ALB,1992,3247039.0,200.852220,0.0


In [47]:
# Calculate total gdp column
df_small['gdp'] = df_small['gdp_per_capita'] * df_small['population']
df_small.head()

,NOC,Year,population,gdp_per_capita,total_medals,gdp
0,AFG,2004,23560654.0,221.763654,0.0,5.224897e+09
1,AFG,2008,26482622.0,381.733238,1.0,1.010930e+10
2,AFG,2012,30560034.0,651.417134,1.0,1.990733e+10
3,AFG,2016,34700612.0,522.082216,0.0,1.811657e+10
4,ALB,1992,3247039.0,200.852220,0.0,6.521750e+08


## Hypothesis Test

Think of the number of medals as a linear equation.

`Total Medals = y-intercept medals + linear-gdp-relationship medals`

Total Medals = $\beta$<sub>0</sub> + $\beta$<sub>1</sub> * GDP<sub>i</sub> + $\epsilon$<sub>i</sub>

### Null Hypothesis:

GDP has no effect on the number of total medals, so $\beta$<sub>1</sub> is 0.

### Alternative Hypothesis

$\beta$<sub>1</sub> $\ne$ 0

Since we're checking for any relationship (on either side of 0), this is a `two-tailed test`.


#### Explore the relationship with a scatterplot

In [48]:
fig = px.scatter(
    df_small,
    x='gdp',
    y='total_medals',
    trendline="ols",
    title='GDP vs Olympic Medal Count'
)
fig.show()

This could potentially be logistic instead of linear.

In [49]:
# Run the regression
X = df_small['gdp']
X = sm.add_constant(X) # Adds intercepts
y = df_small['total_medals']

model = sm.OLS(
    y,
    X
).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           total_medals   R-squared:                       0.324
Model:                            OLS   Adj. R-squared:                  0.321
Method:                 Least Squares   F-statistic:                     117.9
Date:                Fri, 13 Feb 2026   Prob (F-statistic):           1.08e-22
Time:                        11:33:22   Log-Likelihood:                -1156.7
No. Observations:                 248   AIC:                             2317.
Df Residuals:                     246   BIC:                             2324.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.7245      1.695      5.738      0.0

#### Coefficient (GDP): 1.766e-11

This value is very small, but that makes sense because GDP is so big. It's the same as saying 1.706 x 10<sup>-11</sup> X 1_000_000_000 = 0.017 medals per $1 Billion increase in GDP. So, a $100 billion increase in GDP is associated with about 1.7 additional medals.

#### P-Value: 0

Reject the null hypothesis.

If GDP really had no relationship with medals, the probability of observing a coefficient this large (or larger) due to random chance would be less tahn 0.1%

#### R<sup>2</sup>: 0.324

32.4% of the variation in medal counts across country-years is explained by GDP alone.

* This is a moderate explanatory power
* GDP is clearly important
* There's still 67.6% of variation explained by other factors

Other Considerations from Output:
* Prob(JB) = 0.00
* Skew = 3.057
* Kurtosis = 17.065
* Condition number very high

Suggests:
* Non-normal residuals
* Likely skewed GDP distribution
* Potential Scaling issues

Recommendation: log-transform GDP

## Using log(GDP)

In [50]:
# Ensure no 0 gdp values
df_small = df_small[
    df_small['gdp'] > 0
]

# Create log(gdp) column
df_small['log_gdp'] = np.log(df_small['gdp'])
df_small.head()

,NOC,Year,population,gdp_per_capita,total_medals,gdp,log_gdp
0,AFG,2004,23560654.0,221.763654,0.0,5.224897e+09,22.376701
1,AFG,2008,26482622.0,381.733238,1.0,1.010930e+10,23.036721
2,AFG,2012,30560034.0,651.417134,1.0,1.990733e+10,23.714354
3,AFG,2016,34700612.0,522.082216,0.0,1.811657e+10,23.620093
4,ALB,1992,3247039.0,200.852220,0.0,6.521750e+08,20.295823


In [51]:
# Run the new regression model
X = sm.add_constant(df_small['log_gdp'])
y = df_small['total_medals']

model_log = sm.OLS(y, X).fit()
print(model_log.summary())

                            OLS Regression Results                            
Dep. Variable:           total_medals   R-squared:                       0.371
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     144.8
Date:                Fri, 13 Feb 2026   Prob (F-statistic):           1.56e-26
Time:                        11:33:22   Log-Likelihood:                -1147.9
No. Observations:                 248   AIC:                             2300.
Df Residuals:                     246   BIC:                             2307.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -182.1527     16.419    -11.094      0.0

In [52]:
# Compare Models
print("Original R2:", model.rsquared)
print("Log GDP R2:", model_log.rsquared)

Original R2: 0.3239703915321055
Log GDP R2: 0.37052173079750406


Log GDP R2 was higher, so it's a better fit. Even if it's close, interpretation will be cleaner.

In [53]:
# Visualize the fit
fig = px.scatter(
    df_small,
    x="log_gdp",
    y="total_medals",
    trendline="ols",
    title='Log(GDP) vs Olympic Medal Count'
)
fig.show()

Because GDP was highly skewed, a log transformation was applied to improve model fit and interpretability. The log-linear regression indicates a statistically significant positive relationship between economic size and Olympic medal counts (p < 0.001). The transformed model demonstrates improved stability and interpretability compared to the raw GDP specification.

Raw GDP assumed linear effect of dollars which is unrealistic. Log GDP assumes diminishing returns to economic size.

#### Coef (log_gdp) = 8.214

0.01 x 8.214 = 0.082 medals per 1% increase in gdp

This mean a 10% increase in GDP means 0.82 more medals.

If GDP is doubled (100% increase) then 5.7 more medals can be expected.

#### P-Value < 0.001

Reject H<sub>0</sub>

GDP is statistically significantly associated with medal counts.

#### R<sup>2</sup> = 0.371

37.1% of the variation in medal count is explained by log(GDP)

## Adding Population as a Control

Medals = $\beta$<sub>0</sub> + $\beta$<sub>1</sub>log(GDP) + $\beta$<sub>2</sub>log(Population) + $\epsilon$

In [54]:
df_small = df_small[
    df_small['population'] > 0
]

df_small['log_population'] = np.log(df_small['population'])

df_small.head()

,NOC,Year,population,gdp_per_capita,total_medals,gdp,log_gdp,log_population
0,AFG,2004,23560654.0,221.763654,0.0,5.224897e+09,22.376701,16.975089
1,AFG,2008,26482622.0,381.733238,1.0,1.010930e+10,23.036721,17.091999
2,AFG,2012,30560034.0,651.417134,1.0,1.990733e+10,23.714354,17.235204
3,AFG,2016,34700612.0,522.082216,0.0,1.811657e+10,23.620093,17.362268
4,ALB,1992,3247039.0,200.852220,0.0,6.521750e+08,20.295823,14.993254


In [55]:
# Run the multiple regression
X = df_small[[
    'log_gdp',
    'log_population',
]]
X = sm.add_constant(X)

y = df_small['total_medals']

model_full = sm.OLS(y, X).fit()
print(model_full.summary())

                            OLS Regression Results                            
Dep. Variable:           total_medals   R-squared:                       0.377
Model:                            OLS   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                     74.00
Date:                Fri, 13 Feb 2026   Prob (F-statistic):           7.23e-26
Time:                        11:33:22   Log-Likelihood:                -1146.7
No. Observations:                 248   AIC:                             2299.
Df Residuals:                     245   BIC:                             2310.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           -184.7837     16.461    -11.

In [56]:
# Multicollinearity Check
df_small[[
    'log_gdp',
    'log_population',
]].corr()

,log_gdp,log_population
log_gdp,1.00000,0.70394
log_population,0.70394,1.00000


Medals = -184.78 + 7.17 * log(GDP) + 1.72 * log(Population)

#### R<sup>2</sup> = 0.377

0.377 - 0.371 = 0.006

This is a very small improvement and suggests population does not add much to explainability.

#### P-Value of log(Population) = 0.123 > 0.05

Fail to reject the null hypothesis

Population is not statistically significant at the 5% level.

## Poisson Regression

Trying this since medals are counts

Medals = $\beta$<sub>0</sub> + $\beta$<sub>1</sub>X + $\epsilon$

Poisson assumes:

log(E[Medals]) = $\beta$<sub>0</sub> + $\beta$<sub>1</sub>log(GDP) + $\beta$<sub>2</sub>log(Population)

Differences:
* We model the log of expected medal count
* Effects are multiplicative, not additive

In [57]:
# Run the Poisson Regression
X = df_small[[
    'log_gdp',
    'log_population',
]]
X = sm.add_constant(X)

y = df_small['total_medals']

poisson_model = sm.GLM(
    y,
    X,
    family=sm.families.Poisson()
).fit()

print(poisson_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:           total_medals   No. Observations:                  248
Model:                            GLM   Df Residuals:                      245
Model Family:                 Poisson   Df Model:                            2
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2063.9
Date:                Fri, 13 Feb 2026   Deviance:                       3562.0
Time:                        11:46:08   Pearson chi2:                 4.87e+03
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const            -13.0976      0.234    -56.

log(E[Medals])=−13.10+0.585⋅log(GDP)+0.049⋅log(Population)